Learning a new command
=========================
Here the agent reads one example from its own README file and do online research to implement a new command, then try and use it immediately

In [1]:
from lm_agent.agent import Agent
import openai
openai.api_key = 'INVALID_KEY'  # Make sure we don't use OpenAI by accident anywhere
agent = Agent(model='vicuna', work_dir='vicuna', config={'command': {'reader': {'summarization_model': 'vicuna'}}}, disable=('delegate'), essential_only=False)

Loading tokenizer for anon8231489123/vicuna-13b-GPTQ-4bit-128g
Loading GPTQ quantized model...
Loading model ...


/home/virtualzx/miniconda3/envs/ela/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/virtualzx/miniconda3/envs/ela/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/virtualzx/miniconda3/envs/ela/lib/python3.10/site-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will be remov

Done.


### First show the example and ask it to code up the new command


In [2]:
print(agent.instruct("""Read the section 'How to implement a new command' from 
`https://raw.githubusercontent.com/virtualzx-nad/easy_llm_agents/main/README.md` to get the code and instructions for implementing new commands verbatim,
then write a command to get weather forecast for a date in the near future.
When you design your command make sure it does not return too much information to blow up the token limit.
If you choose to use an API, make sure you read the doc to understand the parameters and returns structure, and think about what need to be passed and returned for you command.
I cannot provide you with API keys due to privacy restrictions, but the code can directly retrieve the API keys from `self.metadata`. 
Do not use APIs that are not free, and tell me what API keys need to be passed into metadata if you need them.  
The command will be created or updated as soon as you submit and execute the python code, but make sure to notify me and get approval before you start using them."""))

AI did not use a valid command. Unknown commands: ['weather_forecast']. Resending instructions
<Permissive Overseer>writer requested. "Write a command to get weather forecast for a date in the near future". GRANTED.
<Command writer> {'error': "[Errno 2] No such file or directory: 'How_to_implement_a_new_command.md'", 'filename': 'How_to_implement_a_new_command.md', 'file_type': 'context_file'}
<Command writer> {'filename': 'weather_forecast.py', 'instruction': 'Create a python file that takes a date as an argument and returns the weather forecast for that date', 'context': 'Use the information from the README.md file to create the command'}
<Permissive Overseer>python requested. "Write a python command to get weather forecast for a date in the near future". GRANTED.
<Command python> {'action': 'install_package', 'package': 'search'}
<Command python> {'action': 'install_package', 'package': 'datetime'}
<Command python> {'info': 'Executing code snippet', 'code': 'def weather_forecast(dat

### Add API keys per its request
Add the requested API key to metadata, if the agent requested it.  
There are a few different APIs in this area and the agent usually choose from OpenWeatherMap, WeatherAPI and Open-Meteo APIs. 
If it chose one that is free but needs an API key, it will tell you to supply it here and you need to navigate to the site and get an 
API key then store it into metadata objects below. 
Do not tell the agent what the key is as that is probably not safe behavior.
If it chose one that doesn't need API key (say Open Mateo) you can comment this block out and just proceed to later ones.

In [3]:
# agent.metadata['openweathermap_api_key'] =

### Check if new command is there
Check the command list to make sure the new command is online

In [ ]:
print(agent.generate_command_list())

### Now ask it to run the new command
Now it is in metadata we can tell it to continue.  Then ask the question and see it uses the command

In [ ]:
print(agent.instruct("""OK I have added the API key to the metadata.  
Now you can invoke command `weather_forecast` to check if it works correctly. 
What's the weather in San Francisco the day after tomorrow?"""))

### Under the hood

Now we print out the full conversation history to see what went on under the hood

In [ ]:
for entry in agent.conversation.raw_history:
    print(f"{entry['role']}: {entry['content']}")

### Check how much this has cost. 

Not cheap for a weather forecast, but decent for coding a new functionality

In [ ]:
print(f'Total OpenAI cost: ${agent.model.total_cost():.2f}')